In [ ]:
import pymc as pm
import graphviz
import arviz as az
az.style.use('seaborn')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Generate data

In [ ]:
num_samples = 5_000
random_numbers = np.random.lognormal(mean=0.02, sigma=1, size=num_samples)
default_rates = random_numbers*0.1

In [ ]:
sample_distribution = sns.displot(
    data = default_rates,
    kde = True,
    aspect = 1.5
)

sample_distribution.fig.suptitle('Distribution of Sampled Data')

plt.tight_layout();

### Specify model and *"push inference button"*

In [ ]:
# Prior parameters
num_SMEs = 1_000
default_rate_prior_alpha = 2
default_rate_prior_beta = 5

In [ ]:
# Model definition
with pm.Model() as default_rate_model:
    default_rate = pm.Beta('default_rate', alpha=default_rate_prior_alpha, beta=default_rate_prior_beta)
    defaults = pm.Binomial('defaults', n=num_SMEs, p=default_rate, observed=default_rates)
    trace = pm.sample(15_000, tune=10_500)

In [ ]:
pm.model_to_graphviz(default_rate_model)

### Check posterior predictive

In [ ]:
az.plot_trace(trace);

* stationarity (good! we need this for the `MCMC` simulation to converge...)
* note 4 chains (essentially all lying on top of each other, LHS-plot)

### Summarize posterior

In [ ]:
az.summary(trace)

* see mean value for `default_rate` sampled from posterior distribution (3.7%)
* see value for `r_hat`: the closer it is to `1`, the more the model converges

In [ ]:
# Posterior analysis
az.plot_posterior(trace, var_names=['default_rate']);